<a href="https://colab.research.google.com/github/ychoi-kr/LLM-API/blob/main/langchain/langchain_custom_loader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install openai langchain langchain-community langchain-openai tiktoken

In [ ]:
!pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bi

In [ ]:
#!pip install faiss-gpu
!pip install faiss-cpu

In [ ]:
from google.colab import userdata
import os
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

## 로더 클래스 정의

In [ ]:
from typing import List
import requests

from langchain.docstore.document import Document
from langchain.document_loaders.base import BaseLoader


class WikidocsLoader(BaseLoader):
    def __init__(self, book_id: int, base_url="https://wikidocs.net", **kwargs):
        super().__init__(**kwargs)
        self.book_id = book_id
        self.base_url = base_url
        self.headers = {"Content-Type": "application/json"}

    def load(self) -> List[Document]:
        toc = self._get_toc(self.book_id)
        pages = []
        for item in toc:
            page_id = item["id"]
            page_data = self._get_page(page_id)
            document = Document(
                title=page_data["subject"],
                page_content=page_data["content"],
                metadata={
                    'id': page_id,
                    'source': f"{self.base_url}/{page_id}",
                    'title': page_data["subject"]
                }
            )
            pages.append(document)

        return pages

    def _get_toc(self, book_id):
        url = f"{self.base_url}/api/v1/toc/{book_id}"
        response = requests.get(url, headers=self.headers)
        if response.status_code == 200:
            return response.json()
        else:
            raise ValueError("Failed to get table of contents")

    def _get_page(self, page_id):
        url = f"{self.base_url}/api/v1/page/{page_id}"
        response = requests.get(url, headers=self.headers)
        if response.status_code == 200:
            return response.json()
        else:
            raise ValueError("Failed to get page")

In [ ]:
book_id = 14316  # 생성AI 프로그래밍 트러블슈팅 가이드
loader = WikidocsLoader(book_id)
documents = loader.load()

## 색인 생성

In [ ]:
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

In [ ]:
text_splitter = CharacterTextSplitter(chunk_size=600, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

In [ ]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
search_index = FAISS.from_documents(docs, embeddings)

## 질의응답(Retrieval QA with Sources)

In [ ]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import OpenAI

In [ ]:
retrieval_qa_with_sources_chain = RetrievalQAWithSourcesChain.from_chain_type(
    OpenAI(temperature=0), chain_type="stuff", retriever=search_index.as_retriever()
)

In [ ]:
def retrieval_qa_with_sources(question):
    response = retrieval_qa_with_sources_chain.invoke(
        {"question": question}, return_only_outputs=True
    )
    if response["sources"]:
        return response["answer"] + "출처: " + response["sources"]
    else:
        return response["answer"]

In [ ]:
print(retrieval_qa_with_sources("openai 패키지 구버전과 최신 버전 설치 방법"))

 openai 패키지의 구버전(0.28)과 최신 버전의 설치 방법은 다음과 같다: 
- 구버전(0.28)으로 고정: `pip install -U openai==0.28` (참조: https://wikidocs.net/229554#installing-openai-0.28)
- 최신 버전 설치: `pip install -U openai` (참조: https://wikidocs.net/229554#installing-latest-openai-package)
- 코드 수정하여 신버전의 패키지 사용: openai>=1.0.0에서는 코드를 다음과 같이 수정하면 오류나 경고가 뜨지 않고 잘 실행된다: 
```python
import os
import openai

openai.api_key = os.environ["OPENAI_API_KEY"]

response = openai.[[MARK]]chat.completions[[/MARK]].create(
    model="gpt-3.5-turbo",
    messages=[
        {"role": "user", "content": "hello"},
    ],
)
```
참


In [ ]:
print(retrieval_qa_with_sources("langchain_community.llms.openai.OpenAI 경고가 떠요"))

 The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
출처: https://python.langchain.com/docs/integrations/llms/openai


## 질의응답(QA with sources)

In [ ]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI

In [ ]:
template = """Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES").
If you don't know the answer, just say that you don't know. Don't try to make up an answer.
ALWAYS return a "SOURCES" part in your answer.
Respond in Korean.

QUESTION: {question}
=========
{summaries}
=========
FINAL ANSWER IN KOREAN:"""
PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

qa_with_sources_chain = load_qa_with_sources_chain(
    OpenAI(temperature=0),
    chain_type="stuff",
    prompt=PROMPT
)

In [ ]:
def qa_with_sources(question):
    return qa_with_sources_chain.invoke(
        {
            "input_documents": search_index.similarity_search(question, k=3),
            "question": question,
        },
        return_only_outputs=True,
    )["output_text"]


In [ ]:
print(qa_with_sources('openai 패키지 구버전과 최신 버전 설치 방법'))

 openai 패키지를 설치하는 방법은 두 가지가 있습니다. 첫 번째 방법은 구버전인 0.28로 고정하는 것이고, 두 번째 방법은 최신 버전으로 설치하는 것입니다. 구버전으로 고정하려면 `pip install -U openai==0.28` 명령을 실행하면 됩니다. 최신 버전으로 설치하려면 `pip install -U openai` 명령을 실행하면 됩니다. 하지만 최신 버전에서는 코드를 수정해야 합니다. 따라서 옵션 2를 선택하면 됩니다. 이때 코드를 수정하는 방법은 두 가지가 있습니다. 첫 번째 방법은 다운그레이드하는 것이고, 두 번째 방법은 코드를 수정하는 것입니다. 다운그레이드하는 방법은 [https://wikidocs.net/229554#installing-openai-0.28](https://wikidocs.net


In [ ]:
print(qa_with_sources('langchain_community.llms.openai.OpenAI 경고가 떠요'))



"LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead."라는 경고가 뜹니다. 이는 LangChain 0.1.7에서 `__call__` 함수가 더 이상 사용되지 않으며, 대신 `invoke` 함수를 사용해야 한다는 것을 의미합니다. 따라서 `invoke` 함수를 사용하도록 코드를 수정해야 합니다.

SOURCES: https://wikidocs.net/233334
